# Scanners

In [1]:
from ib_insync import *
util.startLoop()  

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=18)

<IB connected to 127.0.0.1:7497 clientId=18>

In [30]:
import pandas as pd
import datetime as dt
import numpy as np
from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_vollib.black_scholes.greeks.analytical import delta

In [31]:
def winddown(time):
        if time < dt.time(9,30):
            return 0.7
        elif time < dt.time(9,45):
            return 0.6
        elif time < dt.time(10,30):
            return 0.5
        elif time < dt.time(11,30):
            return 0.4
        elif time < dt.time(13):
            return 0.3
        elif time <  dt.time(14,30):
            return 0.25
        elif time < dt.time(15,30):
            return 0.15
        elif time < dt.time(16):
            return 0.05
        else:
            return 0
   
   
def tte(expiry):
    trading_days = np.busday_count( dt.date.today(), expiry )
    winddown_days = winddown(dt.datetime.now().time())
    return (trading_days+winddown_days)/253.0

def bs_implied_vol(F,K,t,kind,price):
    r = 0
    try:
        iv = implied_volatility(price, F,K,t,r, kind.lower())
    except:
        iv = 0
    return iv

def bs_delta(F,K,t,kind,iv):
    r = 0
    try:
        bs_delta = delta(kind.lower(), F,K,t,r, iv)
    except:
        bs_delta = 0
    return bs_delta

## Basic Scanner

To create a scanner, create a `ScannerSubscription` to submit to the `reqScannerData` method. For any scanner to work, at least these three fields must be filled: `instrument` (the what), `locationCode` (the where), and `scanCode` (the ranking).

For example, to find the top ranked US stock percentage gainers:

In [48]:
#Get list of interesting companies

number_of_contracts = 2

sub = ScannerSubscription(
    instrument='STK', 
    locationCode='STK.US.MAJOR', 
    scanCode='TOP_TRADE_COUNT')
scanData = ib.reqScannerData(sub)
list_of_stock_contracts = [i.contractDetails.contract for i in scanData]
list_of_stock_contracts = list_of_stock_contracts[:number_of_contracts]

Error 162, reqId 5: Historical Market Data Service error message:API scanner subscription cancelled: 5


In [32]:
contr = Stock('GME', 'SMART', 'USD')

In [33]:
list_of_stock_contracts = [contr]

In [34]:
# For each company:
# Request ticker for all options

#to do : have better way of selecting strikes to get full coverage but also fewer tickers

ticker_dict = {}
contract_dict = {}
for symbol_contract in list_of_stock_contracts:
    contract_dict[symbol_contract.symbol] = symbol_contract
    ib.qualifyContracts(symbol_contract)
    chains = ib.reqSecDefOptParams(symbol_contract.symbol, '', symbol_contract.secType, symbol_contract.conId)
    util.df(chains)
    chain = next(c for c in chains if c.tradingClass == symbol_contract.symbol and c.exchange == 'SMART')
    print(chain)
    strikes = [strike for strike in chain.strikes
        if strike > 200 and strike < 400]
#         and 0 < strike < amcValue + 20]
    expirations = sorted(exp for exp in chain.expirations)
    rights = ['P', 'C']

    contracts = [Option(symbol_contract.symbol, expiration, strike, right, 'SMART', tradingClass=symbol_contract.symbol)
        for right in rights
        for expiration in expirations
        for strike in strikes]

    contracts = ib.qualifyContracts(*contracts)
    print(len(contracts))
    tickers = ib.reqTickers(*contracts)
    print(tickers[0])
    ticker_dict[symbol_contract.symbol] = tickers
    

OptionChain(exchange='SMART', underlyingConId='36285627', tradingClass='GME', multiplier='100', expirations=['20210611', '20210618', '20210625', '20210702', '20210709', '20210716', '20210723', '20210730', '20210820', '20210917', '20211015', '20211119', '20220121', '20220617', '20230120'], strikes=[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 12.5, 13.0, 14.0, 15.0, 16.0, 17.0, 17.5, 20.0, 21.0, 22.0, 22.5, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0, 125.0, 130.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 152.5, 155.0, 157.5, 160.0, 162.5, 165.0, 167.5, 170.0, 172.5, 175.0, 177.5, 180.0, 182.5, 185.0, 190.0, 195.0, 200.0, 202.5, 205.0, 207.5, 210.0, 212.5

Error 200, reqId 13877: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210611', strike=307.5, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 13879: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210611', strike=312.5, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 13880: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210611', strike=315.0, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 13882: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210611', strike=325.0, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 13884: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='2021

Error 200, reqId 14139: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210716', strike=212.5, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 14140: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210716', strike=215.0, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 14141: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210716', strike=217.5, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 14144: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210716', strike=225.0, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 14143: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='2021

Error 200, reqId 14196: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210723', strike=205.0, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 14197: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210723', strike=207.5, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 14199: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210723', strike=212.5, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 14200: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210723', strike=215.0, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 14203: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='2021

Error 200, reqId 14316: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210820', strike=205.0, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 14317: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210820', strike=207.5, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 14320: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210820', strike=215.0, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 14321: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210820', strike=217.5, right='P', exchange='SMART', tradingClass='GME')
Error 200, reqId 14319: No security definition has been found for the request, contract: Option(symbol='GME', lastTradeDateOrContractMonth='2021

...

Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210917', strike=365.0, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20210917', strike=375.0, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20211015', strike=202.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20211015', strike=205.0, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20211015', strike=207.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20211015', strike=212.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20211015', strike=215.0, right='C', exchange='SMART', tradingClass='GME')

Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20211119', strike=252.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20211119', strike=255.0, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20211119', strike=257.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20211119', strike=262.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20211119', strike=265.0, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20211119', strike=267.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20211119', strike=272.5, right='C', exchange='SMART', tradingClass='GME')

Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20220121', strike=307.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20220121', strike=312.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20220121', strike=315.0, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20220121', strike=325.0, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20220121', strike=335.0, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20220121', strike=345.0, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20220121', strike=355.0, right='C', exchange='SMART', tradingClass='GME')

Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20230120', strike=227.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20230120', strike=232.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20230120', strike=235.0, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20230120', strike=237.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20230120', strike=242.5, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20230120', strike=245.0, right='C', exchange='SMART', tradingClass='GME')
Unknown contract: Option(symbol='GME', lastTradeDateOrContractMonth='20230120', strike=247.5, right='C', exchange='SMART', tradingClass='GME')

994
Ticker(contract=Option(conId=493826564, symbol='GME', lastTradeDateOrContractMonth='20210611', strike=202.5, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GME   210611P00202500', tradingClass='GME'), time=datetime.datetime(2021, 6, 10, 14, 22, 45, 319816, tzinfo=datetime.timezone.utc), bid=0.67, bidSize=24, ask=0.9, askSize=104, last=0.7, lastSize=1, volume=19, high=1.5, low=0.7, close=2.77, halted=0.0, bidGreeks=OptionComputation(impliedVol=2.704224772428763, delta=-0.023454174027517354, optPrice=0.6499999761581421, pvDividend=0.0, gamma=0.0012718868515256036, vega=0.00897515657635517, theta=-0.6499999761581421, undPrice=274.010009765625), askGreeks=OptionComputation(impliedVol=2.9091175190359566, delta=-0.0313037736047413, optPrice=0.8999999761581421, pvDividend=0.0, gamma=0.0015079013955185449, vega=0.012909260958420599, theta=-0.8999999761581421, undPrice=274.010009765625), lastGreeks=OptionComputation(impliedVol=2.604421754086704, delta=-0.0208133

In [35]:
# df with columns : symbol	expiry	strike	kind	und_price	bid_price	ask_price	mid_price	last_price	volume	ib_bid_delta


row_list_for_df = []
for ticker_symbol,ticker_list in ticker_dict.items():
    for ticker in ticker_list:
        if ticker.modelGreeks == None:
            continue
        print(ticker)
        symbol = ticker_symbol
        expiry = dt.datetime.strptime(ticker.contract.lastTradeDateOrContractMonth , '%Y%m%d').date()
        strike = ticker.contract.strike
        kind = ticker.contract.right
        und_price = ticker.modelGreeks.undPrice
        bid_price = ticker.bid
        ask_price = ticker.ask
        mid_price = (bid_price+ask_price)/2
        last_price = ticker.last
        volume = ticker.volume
        ib_bid_delta = ticker.bidGreeks.delta        
        ticker_dict_temp = {'symbol' : symbol , 'expiry' : expiry, 'strike' : strike, 'kind': kind, 'und_price' : und_price,
                       'bid_price': bid_price, 'ask_price' : ask_price, 'mid_price' : mid_price,
                       'last_price' : last_price, 'volume' : volume, 'ib_bid_delta' : ib_bid_delta }
        row_list_for_df.append(ticker_dict_temp)
df = pd.DataFrame(row_list_for_df)

Ticker(contract=Option(conId=493826564, symbol='GME', lastTradeDateOrContractMonth='20210611', strike=202.5, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GME   210611P00202500', tradingClass='GME'), time=datetime.datetime(2021, 6, 10, 14, 22, 45, 319816, tzinfo=datetime.timezone.utc), bid=0.67, bidSize=24, ask=0.9, askSize=104, last=0.7, lastSize=1, volume=19, high=1.5, low=0.7, close=2.77, halted=0.0, bidGreeks=OptionComputation(impliedVol=2.704224772428763, delta=-0.023454174027517354, optPrice=0.6499999761581421, pvDividend=0.0, gamma=0.0012718868515256036, vega=0.00897515657635517, theta=-0.6499999761581421, undPrice=274.010009765625), askGreeks=OptionComputation(impliedVol=2.9091175190359566, delta=-0.0313037736047413, optPrice=0.8999999761581421, pvDividend=0.0, gamma=0.0015079013955185449, vega=0.012909260958420599, theta=-0.8999999761581421, undPrice=274.010009765625), lastGreeks=OptionComputation(impliedVol=2.604421754086704, delta=-0.02081336701

Ticker(contract=Option(conId=495099981, symbol='GME', lastTradeDateOrContractMonth='20210702', strike=287.5, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GME   210702P00287500', tradingClass='GME'), time=datetime.datetime(2021, 6, 10, 14, 23, 0, 15137, tzinfo=datetime.timezone.utc), bid=50.2, bidSize=195, ask=52.8, askSize=171, close=49.01, halted=0.0, bidGreeks=OptionComputation(impliedVol=1.5779518935702486, delta=-0.4712813561173434, optPrice=50.54999923706055, pvDividend=0.0, gamma=0.003734008308455358, vega=0.27056039762833706, theta=-0.9566481165604535, undPrice=274.010009765625), askGreeks=OptionComputation(impliedVol=1.6780785701633445, delta=-0.463406931875252, optPrice=53.54999923706055, pvDividend=0.0, gamma=0.0035046979720669965, vega=0.26987834764725704, theta=-1.0156083635056645, undPrice=274.010009765625), lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=272

Ticker(contract=Option(conId=493828792, symbol='GME', lastTradeDateOrContractMonth='20210709', strike=232.5, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GME   210709P00232500', tradingClass='GME'), time=datetime.datetime(2021, 6, 10, 14, 23, 1, 691185, tzinfo=datetime.timezone.utc), bid=24.25, bidSize=16, ask=26.1, askSize=39, close=26.27, halted=0.0, bidGreeks=OptionComputation(impliedVol=1.506389373243177, delta=-0.2744158810488276, optPrice=24.899999618530273, pvDividend=0.0, gamma=0.0028672607527563884, vega=0.2670063780945071, theta=-0.6696480861382188, undPrice=274.010009765625), askGreeks=OptionComputation(impliedVol=1.5884658074899125, delta=-0.27726719931983723, optPrice=27.25, pvDividend=0.0, gamma=0.002728544416715561, vega=0.26630551115637857, theta=-0.7086760720470374, undPrice=274.010009765625), lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=272.0), modelG

Ticker(contract=Option(conId=468253033, symbol='GME', lastTradeDateOrContractMonth='20210716', strike=340.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GME   210716P00340000', tradingClass='GME'), time=datetime.datetime(2021, 6, 10, 14, 23, 3, 601730, tzinfo=datetime.timezone.utc), bid=100.05, bidSize=63, ask=102.05, askSize=374, volume=1, high=101.47, low=101.47, close=93.75, halted=0.0, bidGreeks=OptionComputation(impliedVol=1.6147133685955437, delta=-0.5676904789807187, optPrice=99.94999694824219, pvDividend=0.0, gamma=0.0028310277769393014, vega=0.3286104013361779, theta=-0.7596158879704829, undPrice=274.0000915527344), askGreeks=OptionComputation(impliedVol=1.6813188438976994, delta=-0.5573162925499, optPrice=102.4000015258789, pvDividend=0.0, gamma=0.00273359529897492, vega=0.32774114973248913, theta=-0.7953648694366603, undPrice=274.0000915527344), lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None

Ticker(contract=Option(conId=496203237, symbol='GME', lastTradeDateOrContractMonth='20210723', strike=335.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GME   210723P00335000', tradingClass='GME'), time=datetime.datetime(2021, 6, 10, 14, 23, 3, 604275, tzinfo=datetime.timezone.utc), bid=99.85, bidSize=30, ask=102.5, askSize=2, close=93.55, halted=0.0, bidGreeks=OptionComputation(impliedVol=1.5744738735176391, delta=-0.5448866399850124, optPrice=99.8499984741211, pvDividend=0.0, gamma=0.002697986231783867, vega=0.38297475324358743, theta=-0.679853635640455, undPrice=272.29998779296875), askGreeks=OptionComputation(impliedVol=1.6607389843452147, delta=-0.5307288980805368, optPrice=102.5, pvDividend=0.0, gamma=0.0025622996642129963, vega=0.38134296841813864, theta=-0.7185287079935324, undPrice=272.29998779296875), lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=272.0), mode

Ticker(contract=Option(conId=467942697, symbol='GME', lastTradeDateOrContractMonth='20220121', strike=240.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GME   220121P00240000', tradingClass='GME'), time=datetime.datetime(2021, 6, 10, 14, 23, 6, 621883, tzinfo=datetime.timezone.utc), bid=85.55, bidSize=63, ask=94.4, askSize=48, close=85.89, halted=0.0, bidGreeks=OptionComputation(impliedVol=1.3246771509633992, delta=-0.2597030906727139, optPrice=85.55000305175781, pvDividend=0.0, gamma=0.0011404424963353636, vega=0.696523140184226, theta=-0.20443174707360168, undPrice=272.57000732421875), askGreeks=OptionComputation(impliedVol=1.4290184455530164, delta=-0.24939888803599242, optPrice=94.4000015258789, pvDividend=0.0, gamma=0.0010339393016885442, vega=0.681047258202895, theta=-0.2159251474441995, undPrice=272.57000732421875), lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=

Ticker(contract=Option(conId=493217888, symbol='GME', lastTradeDateOrContractMonth='20220617', strike=220.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GME   220617P00220000', tradingClass='GME'), time=datetime.datetime(2021, 6, 10, 14, 23, 6, 621883, tzinfo=datetime.timezone.utc), bid=92.85, bidSize=1, ask=105.2, askSize=1, close=94.93, halted=0.0, bidGreeks=OptionComputation(impliedVol=1.3137137190451558, delta=-0.20436047920338118, optPrice=92.8499984741211, pvDividend=0.0, gamma=0.0007816392588380915, vega=0.7868934056172918, theta=-0.138113681515189, undPrice=272.57000732421875), askGreeks=OptionComputation(impliedVol=1.4159310131145764, delta=-0.19314969978317828, optPrice=105.19999694824219, pvDividend=0.0, gamma=0.0006995629617597165, vega=0.7589509057020933, theta=-0.14383799479186027, undPrice=272.57000732421875), lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None, vega=None, theta=None, undPric

Ticker(contract=Option(conId=484813083, symbol='GME', lastTradeDateOrContractMonth='20210611', strike=330.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='GME   210611C00330000', tradingClass='GME'), time=datetime.datetime(2021, 6, 10, 14, 23, 1, 715184, tzinfo=datetime.timezone.utc), bid=1.91, bidSize=2, ask=2.19, askSize=60, last=2.05, lastSize=5, volume=580, high=8.0, low=1.78, close=18.05, halted=0.0, bidGreeks=OptionComputation(impliedVol=2.2991939949262323, delta=0.082763276058901, optPrice=2.0899999141693115, pvDividend=0.0, gamma=0.004183955680977348, vega=0.022597583864082615, theta=-2.0899999141693115, undPrice=271.47930908203125), askGreeks=OptionComputation(impliedVol=2.406143380132825, delta=0.0930479620913643, optPrice=2.4000000953674316, pvDividend=0.0, gamma=0.004372052835884362, vega=0.029948648751459084, theta=-2.4000000953674316, undPrice=271.47930908203125), lastGreeks=OptionComputation(impliedVol=2.3546113271814355, delta=0.08784995580

Ticker(contract=Option(conId=493828573, symbol='GME', lastTradeDateOrContractMonth='20210709', strike=250.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='GME   210709C00250000', tradingClass='GME'), time=datetime.datetime(2021, 6, 10, 14, 23, 12, 762736, tzinfo=datetime.timezone.utc), bid=53.5, bidSize=2, ask=56.4, askSize=42, volume=17, high=61.45, low=55.84, close=83.61, halted=0.0, bidGreeks=OptionComputation(impliedVol=1.445448597053072, delta=0.664702490228353, optPrice=54.75, pvDividend=0.0, gamma=0.003271282124443669, vega=0.27715755622498506, theta=-0.7017674462575083, undPrice=273.5799865722656), askGreeks=OptionComputation(impliedVol=1.5825388593420973, delta=0.6648127696229693, optPrice=59.400001525878906, pvDividend=0.0, gamma=0.0029921098037365905, vega=0.2769347174396657, theta=-0.769515619496349, undPrice=273.5799865722656), lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None, vega=None, theta

Ticker(contract=Option(conId=468253326, symbol='GME', lastTradeDateOrContractMonth='20211119', strike=360.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='GME   211119C00360000', tradingClass='GME'), time=datetime.datetime(2021, 6, 10, 14, 23, 19, 140549, tzinfo=datetime.timezone.utc), bid=64.0, bidSize=15, ask=72.95, askSize=34, close=92.55, halted=0.0, bidGreeks=OptionComputation(impliedVol=1.253283658824946, delta=0.5333059157504964, optPrice=64.0, pvDividend=0.0, gamma=0.001747899758785643, vega=0.729970162621683, theta=-0.2796265177591278, undPrice=272.0), askGreeks=OptionComputation(impliedVol=1.378338684448034, delta=0.5620822025094067, optPrice=72.94999694824219, pvDividend=0.0, gamma=0.001573617795086588, vega=0.7164719041089853, theta=-0.3047245373747516, undPrice=272.0), lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=272.2616882324219), modelGreeks=OptionComput

In [36]:
#add tte, iv and delta
df['tte'] = df.apply(lambda x : tte(x.expiry), axis = 1)
df['implied_vol_mid_point'] = df.apply(lambda x : bs_implied_vol(x.und_price, x.strike, x.tte, x.kind, x.mid_price), axis= 1)
df['mid_point_delta'] =  df.apply(lambda x : bs_delta(x.und_price, x.strike, x.tte, x.kind, x.implied_vol_mid_point), axis = 1)

In [37]:
#restructure df to get call and put in same row
df_calls = df[df.kind == 'C']
df_puts = df[df.kind == 'P']
single_df = pd.merge(df_calls, df_puts, on = ['symbol', 'expiry', 'strike'], suffixes = ("_call", "_put"))
single_df['und_price'] = single_df.und_price_call
single_df['tte'] = single_df.tte_call
single_df = single_df.drop(columns = ['kind_call', 'kind_put', 'und_price_put', 'tte_put', 'und_price_call', 'tte_call' ])

In [38]:
single_df.to_csv('gme_data.csv')

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.


In [ ]:
   ...: 

In [25]:
  greeks_dict['delta'] = ticker.modelGreeks.delta
#         print(type(position), type(ticker.modelGreeks.undPrice) , float(contract.multiplier))
        greeks_dict['cash_delta'] = position * ticker.modelGreeks.delta * ticker.modelGreeks.undPrice * float(contract.multiplier)
        greeks_dict['expiry'] = dt.datetime.strptime(contract.lastTradeDateOrContractMonth , '%Y%m%d').date()
        greeks_dict['strike'] = contract.strike
        greeks_dict['kind'] = contract.right
        greeks_dict['vega'] = ticker.modelGreeks.vega
        greeks_dict['gamma'] = ticker.modelGreeks.gamma
        greeks_dict['theta'] = ticker.modelGreeks.theta
        greeks_dict['weighted_vega'] = GetWeightedVega(ticker.modelGreeks.vega, 1)
        greeks_dict['cash_gamma'] = position * ticker.modelGreeks.gamma * ticker.modelGreeks.undPrice * float(contract.multiplier)
        greeks_dict['avg_cost'] = avg_cost
        greeks_dict['current_price'] = ticker.modelGreeks.optPrice
#     print(greeks_dict)
df = pd.DataFrame({'symbol' : [], 'expiry' : [], 'call_bid_price': [], 'put_bid_price' : [], 
                   'call_offer_price' : []})




chains = ib.reqSecDefOptParams(es.symbol, '', es.secType, es.conId)
# util.df(chains)
# chain = next(c for c in chains if c.tradingClass == es.symbol and c.exchange == 'SMART')
# print(chain)
# strikes = [strike for strike in chain.strikes
#     if strike % 100 == 0]
# #         and 0 < strike < amcValue + 20]
# expirations = sorted(exp for exp in chain.expirations)
# rights = ['P', 'C']

# contracts = [Option(symbol_contract.symbol, expiration, strike, right, 'SMART', tradingClass=symbol_contract.symbol)
#     for right in rights
#     for expiration in expirations
#     for strike in strikes]

# contracts = ib.qualifyContracts(*contracts)
# print(len(contracts))
# tickers = ib.reqTickers(*contracts)
# print(tickers[0])
# # ticker_dict[symbol_contract.symbol] = tickers
    a = Option('ES','20210618', 4000, 'C', 'GLOBEX', tradingClass = 'ES')
b = [a]

In [26]:
rows_list = []
for row in input_rows:

        dict1 = {}
        # get input row in dictionary format
        # key = col_name
        dict1.update(blah..) 

        rows_list.append(dict1)

df = pd.DataFrame(rows_list)    

,symbol,expiry,call_bid_price,put_bid_price,call_offer_price


In [18]:
contract[0]

TypeError: 'Option' object is not subscriptable

In [7]:
list_of_stock_contracts

[Contract(secType='STK', conId=464040647, symbol='CLOV', exchange='SMART', currency='USD', localSymbol='CLOV', tradingClass='NMS'),
 Contract(secType='STK', conId=140070600, symbol='AMC', exchange='SMART', currency='USD', localSymbol='AMC', tradingClass='AMC'),
 Contract(secType='STK', conId=460492611, symbol='WISH', exchange='SMART', currency='USD', localSymbol='WISH', tradingClass='NMS'),
 Contract(secType='STK', conId=131217639, symbol='BB', exchange='SMART', currency='USD', localSymbol='BB', tradingClass='BB'),
 Contract(secType='STK', conId=283766043, symbol='NEXT', exchange='SMART', currency='USD', localSymbol='NEXT', tradingClass='SCM'),
 Contract(secType='STK', conId=90655688, symbol='WEN', exchange='SMART', currency='USD', localSymbol='WEN', tradingClass='NMS'),
 Contract(secType='STK', conId=265598, symbol='AAPL', exchange='SMART', currency='USD', localSymbol='AAPL', tradingClass='NMS'),
 Contract(secType='STK', conId=215119556, symbol='WKHS', exchange='SMART', currency='USD'

*The displayed error 162 can be ignored*

The scanner returns a list of contract details, without current market data (this can be obtained via seperate market data requests).

## Filtering scanner results, the old way

The `ScannerSubscription` object has addional parameters that can be set to filter the results, such as `abovePrice`, `aboveVolume`, `marketCapBelow` or `spRatingAbove`. 

For example, to reuse the previous `sub` and query only for stocks with a price above 200 dollar:

In [3]:
sub.abovePrice = 200
scanData = ib.reqScannerData(sub)

symbols = [sd.contractDetails.contract.symbol for sd in scanData]
print(symbols)

Error 162, reqId 3: Historical Market Data Service error message:API scanner subscription cancelled: 3


['FVRR', 'BA', 'AMGN', 'VRTX', 'LMT']


## Filtering, the new way

In the new way there is a truly vast number of parameters available to use for filtering.
These new scanner parameters map directly to the options available through the TWS "Advanced Market Scanner." The parameters
are dynamically available from a huge XML document that is returned by ``reqScannerParameters``:

In [4]:
xml = ib.reqScannerParameters()

print(len(xml), 'bytes')

Error 162, reqId 4: Historical Market Data Service error message:API scanner subscription cancelled: 4


2000323 bytes


To view the XML in a web browser:

In [5]:
path = 'scanner_parameters.xml'
with open(path, 'w') as f:
    f.write(xml)

import webbrowser
webbrowser.open(path)

True

In [6]:
# parse XML document
import xml.etree.ElementTree as ET
tree = ET.fromstring(xml)

# find all tags that are available for filtering
tags = [elem.text for elem in tree.findall('.//AbstractField/code')]
print(len(tags), 'tags:')
print(tags)

1352 tags:
['priceAbove', 'priceBelow', 'usdPriceAbove', 'usdPriceBelow', 'volumeAbove', 'usdVolumeAbove', 'usdVolumeBelow', 'avgVolumeAbove', 'avgVolumeBelow', 'avgUsdVolumeAbove', 'avgUsdVolumeBelow', 'ihNumSharesInsiderAbove', 'ihNumSharesInsiderBelow', 'ihInsiderOfFloatPercAbove', 'ihInsiderOfFloatPercBelow', 'iiNumSharesInstitutionalAbove', 'iiNumSharesInstitutionalBelow', 'iiInstitutionalOfFloatPercAbove', 'iiInstitutionalOfFloatPercBelow', 'marketCapAbove1e6', 'marketCapBelow1e6', 'moodyRatingAbove', 'moodyRatingBelow', 'spRatingAbove', 'spRatingBelow', 'ratingsRelation', 'bondCreditRating', 'maturityDateAbove', 'maturityDateBelow', 'currencyLike', 'excludeConvertible', 'couponRateAbove', 'couponRateBelow', 'optVolumeAbove', 'optVolumeBelow', 'avgOptVolumeAbove', 'optVolumePCRatioAbove', 'optVolumePCRatioBelow', 'impVolatAbove', 'impVolatBelow', 'impVolatOverHistAbove', 'impVolatOverHistBelow', 'imbalanceAbove', 'imbalanceBelow', 'displayImbalanceAdvRatioAbove', 'displayImbalanc

Notice how ``abovePrice`` is now called ``priceAbove``...

Using three of these filter tags, let's perform a query to find all US stocks that went up 20% and have a current price between 5 and 50 dollar, sorted by percentage gain:

In [7]:
sub = ScannerSubscription(
    instrument='STK',
    locationCode='STK.US.MAJOR',
    scanCode='TOP_PERC_GAIN')

tagValues = [
    TagValue("changePercAbove", "20"),
    TagValue('priceAbove', 5),
    TagValue('priceBelow', 50)]

# the tagValues are given as 3rd argument; the 2nd argument must always be an empty list
# (IB has not documented the 2nd argument and it's not clear what it does)
scanData = ib.reqScannerData(sub, [], tagValues)

symbols = [sd.contractDetails.contract.symbol for sd in scanData]
print(symbols)

[]


Any scanner query that TWS can do can alse be done through the API. The `scanCode` parameter maps directly to the "Parameter" window in the TWS "Advanced Market Scanner." We can verify this by printing out the `scanCode` values available:

In [8]:
scanCodes = [e.text for e in tree.findall('.//scanCode')]

print(len(scanCodes), 'scan codes, showing the ones starting with "TOP":')
print([sc for sc in scanCodes if sc.startswith('TOP')])

762 scan codes, showing the ones starting with "TOP":
['TOP_PERC_GAIN', 'TOP_PERC_LOSE', 'TOP_TRADE_COUNT', 'TOP_TRADE_RATE', 'TOP_PRICE_RANGE', 'TOP_VOLUME_RATE', 'TOP_OPEN_PERC_GAIN', 'TOP_OPEN_PERC_LOSE', 'TOP_AFTER_HOURS_PERC_GAIN', 'TOP_AFTER_HOURS_PERC_LOSE', 'TOP_OPT_IMP_VOLAT_GAIN', 'TOP_OPT_IMP_VOLAT_LOSE', 'TOP_STOCK_BUY_IMBALANCE_ADV_RATIO', 'TOP_STOCK_SELL_IMBALANCE_ADV_RATIO', 'TOP_STOCK_BUY_REG_IMBALANCE_ADV_RATIO', 'TOP_STOCK_SELL_REG_IMBALANCE_ADV_RATIO', 'TOP_PERC_GAIN', 'TOP_PERC_LOSE', 'TOP_TRADE_RATE', 'TOP_PERC_GAIN']


Queries are not limited to stocks. To get a list of all supported instruments:

In [9]:
instrumentTypes = set(e.text for e in tree.findall('.//Instrument/type'))
print(instrumentTypes)

{'BOND.AGNCY', 'ETF.FI.US', 'STOCK.EU', 'SSF.NA', 'BOND.MUNI', 'BOND.GOVT', 'BOND.CD', 'SSF.HK', 'WAR.EU', 'STOCK.HK', 'SSF.EU', 'IND.EU', 'IND.HK', 'FUND.ALL', 'STK', 'FUT.HK', 'BOND', 'FUT.US', 'STOCK.NA', 'BOND.GOVT.NON-US', 'PORTSTK', 'Global', 'IND.US', 'FUT.EU', 'STOCK.ME', 'SLB.US', 'NATCOMB', 'FUT.NA', 'ETF.EQ.US'}


To find all location codes:

In [10]:
locationCodes = [e.text for e in tree.findall('.//locationCode')]
print(locationCodes)

['STK.US', 'STK.US.MAJOR', 'STK.NYSE', 'STK.AMEX', 'STK.ARCA', 'STK.NASDAQ', 'STK.NASDAQ.NMS', 'STK.NASDAQ.SCM', 'STK.BATS', 'STK.US.MINOR', 'STK.PINK', 'STK.OTCBB', 'ETF.EQ.US', 'ETF.EQ.US.MAJOR', 'ETF.EQ.ARCA', 'ETF.EQ.NASDAQ.NMS', 'ETF.EQ.BATS', 'ETF.FI.US', 'ETF.FI.US.MAJOR', 'ETF.FI.ARCA', 'ETF.FI.NASDAQ.NMS', 'ETF.FI.BATS', 'FUT.US', 'FUT.GLOBEX', 'FUT.ECBOT', 'FUT.IPE', 'FUT.NYBOT', 'FUT.NYMEX', 'FUT.NYSELIFFE', 'FUT.CFE', 'FUT.CFECRYPTO', 'FUT.CMECRYPTO', 'FUT.ICECRYPTO', 'IND.US', 'BOND.WW', 'BOND.US', 'BOND.EU.EURONEXT', 'BOND.CD.US', 'BOND.AGNCY.US', 'BOND.GOVT.US', 'BOND.MUNI.US', 'BOND.GOVT.NON-US', 'BOND.GOVT.US.NON-US', 'BOND.GOVT.EU.EURONEXT', 'BOND.GOVT.HK.SEHK', 'SLB.PREBORROW', 'FUND.ALL', 'FUND.US', 'FUND.NON-US', 'STK.NA', 'STK.NA.CANADA', 'STK.NA.TSE', 'STK.NA.VENTURE', 'STK.NA.MEXI', 'FUT.NA', 'FUT.NA.CDE', 'FUT.NA.MEXDER', 'SSF.NA', 'SSF.NA.MEXDER', 'STK.EU', 'STK.EU.VSE', 'STK.EU.SBF', 'STK.EU.IBIS', 'STK.EU.IBIS-XETRA', 'STK.EU.IBIS-EUSTARS', 'STK.EU.IBIS-USST

In [11]:
ib.disconnect()